# How to convert Solvency 2 XBRL-instances to HTML and CSV-files

This tutorial shows you how to convert a XBRL-instance of a Solvency 2 report to a csv-file. This enables you to use the data from the XBRL-instance in your Python environment.

* install the Arelle package

* download the Solvency 2 taxonomy from the EIOPA website

* get your XBRL-instance(s) (we use here the Solvency 2 XBRL instance examples from the EIOPA website)

* read the XBRL-instance and convert it to a csv-file

## Install the Arelle package

Clone the package from https://github.com/Arelle/Arelle.git and install the package.

## Download the Solvency 2 taxonomy from the EIOPA website

The file we downloaded was:
https://dev.eiopa.europa.eu/Taxonomy/Full/2.4.0/S2/EIOPA_SolvencyII_XBRL_Taxonomy_2.4.0_with_external_hotfix.zip

## Get your XBRL-instance(s)

We got ours from:
https://eiopa.europa.eu/regulation-supervision/insurance/reporting-format/data-point-model-and-xbrl. 

The test instances are here:
https://dev.eiopa.europa.eu/Taxonomy/Full/2.4.0/S2/EIOPA_SolvencyII_XBRL_Instance_documents_2.4.0.zip


## Read the XBRL-instance

In [1]:
from arelle import ViewFileFactTable, ViewFileFactList, ModelManager, FileSource, ViewFileRelationshipSet
from arelle import Cntlr, ModelXbrl, ModelDocument, ViewFileFormulae
from arelle import ModelFormulaObject, RenderingEvaluator, XbrlConst
from arelle.ModelDtsObject import ModelRelationship
import arelle
from arelle.ModelRenderingObject import (ModelEuTable, ModelTable, ModelBreakdown,
                                         ModelEuAxisCoord, ModelDefinitionNode, ModelClosedDefinitionNode, ModelRuleDefinitionNode,
                                         ModelRelationshipDefinitionNode, ModelSelectionDefinitionNode, ModelFilterDefinitionNode,
                                         ModelConceptRelationshipDefinitionNode, ModelDimensionRelationshipDefinitionNode,
                                         ModelCompositionDefinitionNode, ModelTupleDefinitionNode, StructuralNode,
                                         ROLLUP_NOT_ANALYZED, CHILDREN_BUT_NO_ROLLUP, CHILD_ROLLUP_FIRST, CHILD_ROLLUP_LAST,
                                         OPEN_ASPECT_ENTRY_SURROGATE)
from arelle.ModelDtsObject import ModelResource

In [2]:
import os
import src

In [3]:
# make sure you have a 'arelle' directory in the data_path! (This is where the taxonomy is stored)
DATA_PATH    = 'C:\\Users\\wjwil\\20_local_data\\xbrl\\' 
RESULTS_PATH = 'C:\\Users\\wjwil\\50_results\\xbrl\\'
LANGUAGE     = "en-GB"
# set the location of taxonomy
os.environ['XDG_CONFIG_HOME'] = DATA_PATH 

In [4]:
# Now we make a modelmanager
modelmanager = ModelManager.initialize(Cntlr.Cntlr())
modelmanager.defaultLang = LANGUAGE
modelmanager.formulaOptions = ModelFormulaObject.FormulaOptions()
modelmanager.loadCustomTransforms()

In [5]:
# And we read the XBRL instance
XBRL_INSTANCE = 'qrs_240_instance.xbrl'

xbrl_instance = ModelXbrl.load(modelmanager, DATA_PATH + XBRL_INSTANCE)

## Convert XBRL-instance to HTML

In [ ]:
tables = list(xbrl_instance.relationshipSet("Table-rendering").linkRoleUris)
tables.sort()
RenderingEvaluator.init(xbrl_instance)
for item in list(tables):
    arelle.ViewFileRenderedGrid.viewRenderedGrid(xbrl_instance,
                 RESULTS_PATH + item[-13:]+".html", 
                 viewTblELR = item,
                 lang = LANGUAGE, 
                 sourceView = None,
                 diffToFile = False)

## XBRL RelationshipSet

In [ ]:
# arcole can be one of the following:
# XbrlConst.parentChild
# XbrlConst.summationItem
# "XBRL-dimensions"
# "Table-rendering"
# isinstance(arcrole, (list,tuple)) 
# XbrlConst.isResourceArcrole(arcrole):

RenderingEvaluator.init(xbrl_instance)
ViewFileRelationshipSet.viewRelationshipSet(modelXbrl = xbrl_instance, 
                                            arcrole = XbrlConst.parentChild,
                                            header = "test",
                                            outfile = RESULTS_PATH + "parentchild.html", 
                                            lang = "en-GB") 

In [ ]:
# a = src.processXBRL.DartXbrl(xbrl = xbrl_instance)
# labels = []
# for table in a.tables:
#     labels.append((table.uri, table.labels))

In [ ]:
arcrole = XbrlConst.parentChild
relationship = xbrl_instance.relationshipSet(arcrole)

parentchildrelations = None
if relationship is not None:
    parentchildrelations = []
    for uri in relationship.linkRoleUris:
        role_types = xbrl_instance.roleTypes.get(uri)
        if role_types is not None:
            definition = role_types[0].genLabel(lang='en-GB')
        else:
            definition = uri
        code = uri
        parentchildrelations.append(src.processXBRL.Table(xbrl_instance, xbrl_instance, code = code, definition = definition, uri = uri))

## Convert XBRL-instance to DataFrame

TODO: z-axis tables

In [6]:
tables = list(xbrl_instance.relationshipSet("Table-rendering").linkRoleUris)
tables.sort()

RenderingEvaluator.init(xbrl_instance)
for table in tables:
    src.toDataFrame.generateCSVTables(xbrl_instance, RESULTS_PATH, table_uri = table, lang = LANGUAGE)

Formula xpath2 grammar initialized in 0,85 secs


Closed definition node s2md_c74 does not contribute at least one structural node
Closed definition node s2md_c499 does not contribute at least one structural node
Closed definition node s2md_c2703 does not contribute at least one structural node
Closed definition node s2md_c2707 does not contribute at least one structural node
Closed definition node s2md_c2714 does not contribute at least one structural node
Closed definition node s2md_c4300 does not contribute at least one structural node
Closed definition node s2md_c8902 does not contribute at least one structural node
Closed definition node s2md_c8912 does not contribute at least one structural node
Closed definition node s2md_c8968 does not contribute at least one structural node


In [ ]:
# Experiments

arcrole = "Table-rendering"
relationship = xbrl_instance.relationshipSet(arcrole)

tables = None
if relationship is not None:
    tables = []
    for role_uri in relationship.linkRoleUris:
        definition = os.path.basename(role_uri)
    #    tables.append(src.processXBRL.Table(xbrl_instance, xbrl_instance, code = uri, definition = definition, uri = uri))
        
        tblAxisRelSet = xbrl_instance.relationshipSet(XbrlConst.euTableAxis, role_uri)
        if len(tblAxisRelSet.modelRelationships) > 0:
            axisSubtreeRelSet = xbrl_instance.relationshipSet(XbrlConst.euAxisMember, role_uri)
        else:
            tblAxisRelSet = xbrl_instance.relationshipSet((XbrlConst.tableBreakdown, XbrlConst.tableBreakdownMMDD, XbrlConst.tableBreakdown201305, XbrlConst.tableBreakdown201301, XbrlConst.tableAxis2011), role_uri)
            axisSubtreeRelSet = xbrl_instance.relationshipSet((XbrlConst.tableBreakdownTree, XbrlConst.tableBreakdownTreeMMDD, XbrlConst.tableBreakdownTree201305, XbrlConst.tableDefinitionNodeSubtree, XbrlConst.tableDefinitionNodeSubtreeMMDD, XbrlConst.tableDefinitionNodeSubtree201305, XbrlConst.tableDefinitionNodeSubtree201301, XbrlConst.tableAxisSubtree2011), role_uri)

        for rootconcept in tblAxisRelSet.rootConcepts:
            # rootconcept is a modelTable-object
            print(rootconcept.definitionLabelsView)
            tblAxisRels = tblAxisRelSet.fromModelObject(rootconcept)
            for i, tblAxisRel in enumerate(tblAxisRels):
                definitionNode = tblAxisRel.toModelObject
                print(tblAxisRel.axisDisposition + ": ", end = "")
                for rel in xbrl_instance.relationshipSet(XbrlConst.elementLabel).fromModelObject(definitionNode):
                    if isinstance(rel.toModelObject, ModelResource) and rel.toModelObject.role != XbrlConst.genStandardLabel:
                        labelRole = rel.toModelObject.role
                        print("Non-standard label role : " + labelRole)
                    if isinstance(rel.toModelObject, ModelResource) and rel.toModelObject.role == XbrlConst.genStandardLabel:
                        labelRole = rel.toModelObject.role
                        print("Standard label role : " + labelRole)
        print('')

## List validation rules in taxonomy

In [6]:
import pandas as pd

In [7]:
ViewFileFormulae.viewFormulae(xbrl_instance, RESULTS_PATH + "formulae.csv", "header", None)

In [16]:
formulae = pd.read_csv(RESULTS_PATH + "formulae.csv")
formulae[formulae['Expression'].str[0:2]=="BV"]['Expression'].values

array(['BV916-1: if ({{S.28.01.01.05,r0400,c0070}} != {{S.28.01.01.05,r0350,c0070}}) then {{S.28.01.01.05,r0400,c0070}} >= 0.25*{{S.28.01.01.05,r0310,c0070}} and {{S.28.01.01.05,r0400,c0070}} <= 0.45*{{S.28.01.01.05,r0310,c0070}}',
       'BV916-1: if ({{S.28.01.01.05,r0400,c0070}} != {{S.28.01.01.05,r0350,c0070}}) then {{S.28.01.01.05,r0400,c0070}} >= 0.25*{{S.28.01.01.05,r0310,c0070}} and {{S.28.01.01.05,r0400,c0070}} <= 0.45*{{S.28.01.01.05,r0310,c0070}}',
       'BV916-1: if ({{S.28.01.01.05,r0400,c0070}} != {{S.28.01.01.05,r0350,c0070}}) then {{S.28.01.01.05,r0400,c0070}} >= 0.25*{{S.28.01.01.05,r0310,c0070}} and {{S.28.01.01.05,r0400,c0070}} <= 0.45*{{S.28.01.01.05,r0310,c0070}}',
       ...,
       'BV921-1: {{S.28.02.01.05, r0330,c0130}} = 0.25 * {{S.28.02.01.05, r0310,c0130}}',
       'BV920-1_W: {{S.28.02.01.05, r0320,c0130}} = 0.45 * {{S.28.02.01.05, r0310,c0130}}',
       'BV920-1_W: {{S.28.02.01.05, r0320,c0130}} = 0.45 * {{S.28.02.01.05, r0310,c0130}}'],
      dtype=objec

In [17]:
len(formulae[formulae['Expression'].str[0:2]=="BV"]['Expression'])

1210

In [ ]:
formulae

In [ ]:
# from arelle import ViewFileFactTable
# ViewFileFactTable.viewFacts(xbrl_instance, RESULTS_PATH + "facts.html", arcrole=None, linkrole=None, linkqname=None, arcqname=None, ignoreDims=False, showDimDefaults=False, labelrole=None, lang = "en-GB")

In [ ]:
# from arelle import ViewFileFactList
# ViewFileFactTable.viewFacts(xbrl_instance, RESULTS_PATH + "facts_list.html")

In [ ]:
import regex
from src import saveHtmlEBAtables, saveLoadableExcel, saveSampleInstance, saveLoadableOIM

In [ ]:
saveLoadableOIM.saveLoadableOIM(xbrl_instance, "test.csv")